<a href="https://colab.research.google.com/github/Cordero27/Minor-Machine-Learning/blob/main/Tarea_Final_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tarea Final - Machine Learning (Minor Ciencias de Datos)**

###Para ejecutar correctamente es código es necesario disponer del conjunto de datos WISDM: WIreless Sensor Data Mining, el cual puede ser descargado de en el siguente [enlace directo de descarga](https://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz) (es posible que su acceso desde Github este restringido, por lo que se recomienda abrir el enlace desde Google Colab).


###Dentro del fichero .tar.gz (archivo descargado anteriormente) se encuentra un archivo de formato .txt llamado WISDM_ar_v1.1_raw, que posee los datos a estudiar en el presente ejercicio.

###Para simplificar el acceso a estos datos desde el Jupyter Notebook decidió subir este archivo a Github, por lo que podemos acceder a su contenido disponiendo tan solo de su url.


In [157]:
#Librerias a utilizar
import pandas as pd

In [158]:
#Fuente del dataset: https://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz
#Afortunadamente también podemos usar la función de extración de datos para archivos .csv
#Como este archivo no posee cabeceras, las incorporamos a la hora de leerlo con el parámetro names.
#Además, como el archivo con los datos posee filas de datos incompletas, recurrimos a ignorarlas para evitar alterar las estadisticas del conjunto con datos externos.
data = pd.read_csv('https://raw.githubusercontent.com/Cordero27/Minor-Machine-Learning/main/WISDM_ar_v1.1_raw.txt', on_bad_lines='skip',
                   names=["user","activity","timestamp","x-acceleration","y-accel","z-accel"])

In [159]:
#Observamos el contenido adquirido del archivo .txt
data.head()

,user,activity,timestamp,x-acceleration,y-accel,z-accel
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286;
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433;
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209;
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172;
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164;


##Fase de preprocesamiento:

###Ejecutamos paso a paso la limpieza de los datos. Cabe destacar que, al importar el conjunto de datos, se tuvo que eliminar las filas con datos faltantes, pues no hacerlo hace imposible almacenar el conjunto en un DataFrame.


In [160]:
#Verificamos si existen NAs dentro del conjunto de datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098204 entries, 0 to 1098203
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   user            1098204 non-null  int64  
 1   activity        1098204 non-null  object 
 2   timestamp       1098204 non-null  int64  
 3   x-acceleration  1098204 non-null  float64
 4   y-accel         1098204 non-null  float64
 5   z-accel         1098203 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 50.3+ MB


In [161]:
#Como en la columna z-accel tiene un valor null, debemos equilibrar el resto de columnas.
dataWONA = data.dropna()
dataWONA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098203 entries, 0 to 1098203
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   user            1098203 non-null  int64  
 1   activity        1098203 non-null  object 
 2   timestamp       1098203 non-null  int64  
 3   x-acceleration  1098203 non-null  float64
 4   y-accel         1098203 non-null  float64
 5   z-accel         1098203 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 58.7+ MB


In [162]:
#Eliminamos la columna user, ya que no es relevante en este caso de estudio.
dataWONA = dataWONA.drop(dataWONA.columns[[0]], axis=1)
dataWONA.head()

,activity,timestamp,x-acceleration,y-accel,z-accel
0,Jogging,49105962326000,-0.694638,12.680544,0.50395286;
1,Jogging,49106062271000,5.012288,11.264028,0.95342433;
2,Jogging,49106112167000,4.903325,10.882658,-0.08172209;
3,Jogging,49106222305000,-0.612916,18.496431,3.0237172;
4,Jogging,49106332290000,-1.184970,12.108489,7.205164;


In [163]:
#Notamos que en la columna z-accel el caracter ';' esta presente al final de cada numero, por lo cual debemos quitarlo
dataWONA['z-accel'] = dataWONA['z-accel'].str.rstrip(';')
dataWONA.head()

,activity,timestamp,x-acceleration,y-accel,z-accel
0,Jogging,49105962326000,-0.694638,12.680544,0.50395286
1,Jogging,49106062271000,5.012288,11.264028,0.95342433
2,Jogging,49106112167000,4.903325,10.882658,-0.08172209
3,Jogging,49106222305000,-0.612916,18.496431,3.0237172
4,Jogging,49106332290000,-1.184970,12.108489,7.205164


In [164]:
#Como desde un inicio, la columna z-accel tenia el caracter ';', el sistema consideró la fila con tipo object, por lo tanto debemos pasarla a tipo float
#dataWONA['z-accel'] = dataWONA['z-accel'].astype(str).astype(float)
#NOS ENCONTRAMOS CON UN ERROR!!, pero nos permite conocer que la fila z-accel aun no está limpia
#(ESTA COMENTADA LA LINEA DE CÓDIGO PARA EVITAR QUE LA EJECUCIÓN AUTOMÁTICA DE CELDAS SE CANCELE PRODUCTO DE ESTE ERROR)

In [165]:
#Con el error anterior, sabemos que existen ';' entre los numeros de la columna, entonces reemplazamos el ";"" por "".
import re
dataWONA['z-accel']=dataWONA['z-accel'].str.replace(";","")

In [166]:
#Finalmente, ya podemos cambiar el tipado de la columna, y aprovechamos de pasar la otra columna tipo object (activity) por category
dataWONA['z-accel'] = dataWONA['z-accel'].astype(str).astype(float)
dataWONA.loc[:, ("activity")] = pd.Categorical(dataWONA.loc[:, ("activity")])
dataWONA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098203 entries, 0 to 1098203
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype   
---  ------          --------------    -----   
 0   activity        1098203 non-null  category
 1   timestamp       1098203 non-null  int64   
 2   x-acceleration  1098203 non-null  float64 
 3   y-accel         1098203 non-null  float64 
 4   z-accel         1098203 non-null  float64 
dtypes: category(1), float64(3), int64(1)
memory usage: 42.9 MB


In [167]:
#Finalmente, transformamos todas las columnas a un tipado mas liviano, dejando al conjunto de datos con un peso de 16.8 MB, considerando que antes 
#pesaba 50.3 MB. Además, convertimos los valores de la columna activity en enteros, para trabajar de forma mas comoda y rápida (ojo, sigue siendo una 
#columna categorica).
import numpy as np
dataWONA.loc[:, ("activity")] = dataWONA.activity.cat.codes
dataWONA['activity'] = dataWONA['activity'].astype(np.uint8)
dataWONA['timestamp'] = dataWONA['timestamp'].astype(np.uint8)
dataWONA['x-acceleration'] = dataWONA['x-acceleration'].astype(np.float16)
dataWONA['y-accel'] = dataWONA['y-accel'].astype(np.float16)
dataWONA['z-accel'] = dataWONA['z-accel'].astype(np.float16)
dataWONA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098203 entries, 0 to 1098203
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   activity        1098203 non-null  uint8  
 1   timestamp       1098203 non-null  uint8  
 2   x-acceleration  1098203 non-null  float16
 3   y-accel         1098203 non-null  float16
 4   z-accel         1098203 non-null  float16
dtypes: float16(3), uint8(2)
memory usage: 16.8 MB


In [168]:
#Notamos que el conjunto de datos, luego de la limpieza, está igual que antes, pero con 
dataWONA.head()

,activity,timestamp,x-acceleration,y-accel,z-accel
0,1,240,-0.694824,12.679688,0.503906
1,1,24,5.011719,11.265625,0.953613
2,1,88,4.902344,10.882812,-0.081726
3,1,232,-0.612793,18.500000,3.023438
4,1,208,-1.184570,12.109375,7.207031
